# Live Data Reduction


## From Terminal

You can run the ``beamlime`` command from your terminal to run the application and see the real-time update of a plot.

```bash
beamlime --image-path beamlime_plot.png
```

This command will save the plot into ``beamlime_plot.png``.

## Jupyter Environment

You can also run the same thing in jupyter environment
and use ``plopp`` matplotlib widget to see the real-time updated plot.

**It uses the same ``PlotHandler`` as above,**
**so running this application will create a file called ``beamlime_plot.png`` in the current directory.**

In [ ]:
%matplotlib widget
import scipp as sc

sc.get_logger().addHandler(sc.logging.make_widget_handler())
sc.display_logs()

In [ ]:
from beamlime.constructors import SingletonProvider, Factory, multiple_constant_providers
from beamlime.applications.handlers import (
    DataReductionHandler,
    WorkflowResultUpdate,
    PlotStreamer,
    DetectorDataReceived,
)
from beamlime.applications.daemons import (
    DetectorDataReceived,
    FakeListener,
    NexusTemplatePath,
    RunStart,
)
from beamlime.applications.base import Application
from beamlime.executables.prototypes import collect_default_providers
from beamlime.logging import BeamlimeLogger
from beamlime.applications._parameters import NumFrames, DataFeedingSpeed
from beamlime.stateless_workflow import Workflow

# Build Factory
prototype_factory = Factory(collect_default_providers())
prototype_factory.providers[FakeListener] = FakeListener
prototype_factory.providers[PlotStreamer] = SingletonProvider(PlotStreamer)
parameters = {
    BeamlimeLogger: sc.get_logger(),
    DataFeedingSpeed: DataFeedingSpeed(1/14),
    NumFrames: NumFrames(5),
    Workflow: Workflow('dummy'),
    NexusTemplatePath: NexusTemplatePath('../../tests/applications/ymir.json')
}

In [ ]:
from matplotlib import pyplot as plt
with multiple_constant_providers(prototype_factory, parameters):
    # Build the application
    prototype_factory[BeamlimeLogger].setLevel("INFO")
    app = prototype_factory[Application]
    # Register Handlers
    plot_streamer = prototype_factory[PlotStreamer]
    app.register_handling_method(WorkflowResultUpdate, plot_streamer.update_histogram)
    data_reduction_handler = prototype_factory[DataReductionHandler]
    app.register_handling_method(RunStart, data_reduction_handler.update_nexus_template)
    app.register_handling_method(DetectorDataReceived, data_reduction_handler.process_message)
    # Register Daemons
    app.register_daemon(prototype_factory[FakeListener])

    app.run()
    plt.close()  # Close the plot from this cell.

In [ ]:
from plopp.widgets import Box, HBar

Box(HBar(list(plot_streamer.figures.values())))